Notebook creado para mostrar algunas de las formas de llevar a cabo joins entre DataFrames aplicando a su vez operaciones de ventana sobre los datos

In [2]:
import org.apache.spark.sql.types.{StringType, StructField, IntegerType, DateType, StructType}

val salaries_file_location = "../data/salaries.csv"
val employees_file_location = "../data/employees.csv"
val departments_file_location = "../data/departments.csv"
val dept_emp_file_location = "../data/dept_emp.csv"


import org.apache.spark.sql.Row
val salariesSchema = StructType(
  Array(
    StructField("emp_no", IntegerType, true),
    StructField("salary", IntegerType, true),
    StructField("from_date", DateType, true),
    StructField("to_date", DateType, true)
  )
)

val employeesSchema = StructType(
  Array(
    StructField("emp_no", IntegerType, true),
    StructField("birth_date", DateType, true),
    StructField("first_name", StringType, true),
    StructField("last_name", StringType, true),
    StructField("gender", StringType, true),
    StructField("hire_date", DateType, true)
  )
)

val departmentsSchema = StructType(
  Array(
    StructField("dept_no", StringType, true),
    StructField("dept_name", StringType, true)
  )
)

val deptEmpSchema = StructType(
  Array(
    StructField("emp_no", IntegerType, true),
    StructField("dept_no", StringType, true),
    StructField("from_date", DateType, true),
    StructField("to_date", DateType, true)
  )
)
  

val salariesDF = spark.read.format("csv")
.option("inferSchema", false)
.option("header", true)
.option("ignoreLeadingWhiteSpace", true)
.option("ignoreTrailingWhiteSpace", true)
.option("sep", ",")
.option("quote", "\"")
.option("escape", "\"")
.schema(salariesSchema)
.load(salaries_file_location).cache()

val employeesDF = spark.read.format("csv")
.option("inferSchema", false)
.option("header", true)
.option("ignoreLeadingWhiteSpace", true)
.option("ignoreTrailingWhiteSpace", true)
.option("sep", ",")
.option("quote", "\"")
.option("escape", "\"")
.schema(employeesSchema)
.load(employees_file_location).cache()

val departmentsDF = spark.read.format("csv")
.option("inferSchema", false)
.option("header", true)
.option("ignoreLeadingWhiteSpace", true)
.option("ignoreTrailingWhiteSpace", true)
.option("sep", ",")
.option("quote", "\"")
.option("escape", "\"")
.schema(departmentsSchema)
.load(departments_file_location).cache()

val deptEmpDF = spark.read.format("csv")
.option("inferSchema", false)
.option("header", true)
.option("ignoreLeadingWhiteSpace", true)
.option("ignoreTrailingWhiteSpace", true)
.option("sep", ",")
.option("quote", "\"")
.option("escape", "\"")
.schema(deptEmpSchema)
.load(dept_emp_file_location).cache()

import org.apache.spark.sql.types.{StringType, StructField, IntegerType, DateType, StructType}
salaries_file_location: String = ../data/salaries.csv
employees_file_location: String = ../data/employees.csv
departments_file_location: String = ../data/departments.csv
dept_emp_file_location: String = ../data/dept_emp.csv
import org.apache.spark.sql.Row
salariesSchema: org.apache.spark.sql.types.StructType = StructType(StructField(emp_no,IntegerType,true), StructField(salary,IntegerType,true), StructField(from_date,DateType,true), StructField(to_date,DateType,true))
employeesSchema: org.apache.spark.sql.types.StructType = StructType(StructField(emp_no,IntegerType,true), StructField(birth_date,DateType,true), StructField(first_name,StringType,true), StructField(last_name,StringType,true), Str...


In [3]:
salariesDF.show(false)

+------+------+----------+----------+
|emp_no|salary|from_date |to_date   |
+------+------+----------+----------+
|10001 |60117 |1986-06-26|1987-06-26|
|10001 |62102 |1987-06-26|1988-06-25|
|10001 |66074 |1988-06-25|1989-06-25|
|10001 |66596 |1989-06-25|1990-06-25|
|10001 |66961 |1990-06-25|1991-06-25|
|10001 |71046 |1991-06-25|1992-06-24|
|10001 |74333 |1992-06-24|1993-06-24|
|10001 |75286 |1993-06-24|1994-06-24|
|10001 |75994 |1994-06-24|1995-06-24|
|10001 |76884 |1995-06-24|1996-06-23|
|10001 |80013 |1996-06-23|1997-06-23|
|10001 |81025 |1997-06-23|1998-06-23|
|10001 |81097 |1998-06-23|1999-06-23|
|10001 |84917 |1999-06-23|2000-06-22|
|10001 |85112 |2000-06-22|2001-06-22|
|10001 |85097 |2001-06-22|2002-06-22|
|10001 |88958 |2002-06-22|9999-01-01|
|10002 |65828 |1996-08-03|1997-08-03|
|10002 |65909 |1997-08-03|1998-08-03|
|10002 |67534 |1998-08-03|1999-08-03|
+------+------+----------+----------+
only showing top 20 rows



In [4]:
employeesDF.show(false)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|last_name  |gender|hire_date |
+------+----------+----------+-----------+------+----------+
|10001 |1953-09-02|Georgi    |Facello    |M     |1986-06-26|
|10002 |1964-06-02|Bezalel   |Simmel     |F     |1985-11-21|
|10003 |1959-12-03|Parto     |Bamford    |M     |1986-08-28|
|10004 |1954-05-01|Chirstian |Koblick    |M     |1986-12-01|
|10005 |1955-01-21|Kyoichi   |Maliniak   |M     |1989-09-12|
|10006 |1953-04-20|Anneke    |Preusig    |F     |1989-06-02|
|10007 |1957-05-23|Tzvetan   |Zielinski  |F     |1989-02-10|
|10008 |1958-02-19|Saniya    |Kalloufi   |M     |1994-09-15|
|10009 |1952-04-19|Sumant    |Peac       |F     |1985-02-18|
|10010 |1963-06-01|Duangkaew |Piveteau   |F     |1989-08-24|
|10011 |1953-11-07|Mary      |Sluis      |F     |1990-01-22|
|10012 |1960-10-04|Patricio  |Bridgland  |M     |1992-12-18|
|10013 |1963-06-07|Eberhardt |Terkki     |M     |1985-10-20|
|10014 |1956-02-12|Berni

In [5]:
departmentsDF.show(false)

+-------+------------------+
|dept_no|dept_name         |
+-------+------------------+
|d009   |Customer Service  |
|d005   |Development       |
|d002   |Finance           |
|d003   |Human Resources   |
|d001   |Marketing         |
|d004   |Production        |
|d006   |Quality Management|
|d008   |Research          |
|d007   |Sales             |
+-------+------------------+



In [6]:
deptEmpDF.show(false)

+------+-------+----------+----------+
|emp_no|dept_no|from_date |to_date   |
+------+-------+----------+----------+
|10001 |d005   |1986-06-26|9999-01-01|
|10002 |d007   |1996-08-03|9999-01-01|
|10003 |d004   |1995-12-03|9999-01-01|
|10004 |d004   |1986-12-01|9999-01-01|
|10005 |d003   |1989-09-12|9999-01-01|
|10006 |d005   |1990-08-05|9999-01-01|
|10007 |d008   |1989-02-10|9999-01-01|
|10008 |d005   |1998-03-11|2000-07-31|
|10009 |d006   |1985-02-18|9999-01-01|
|10010 |d004   |1996-11-24|2000-06-26|
|10010 |d006   |2000-06-26|9999-01-01|
|10011 |d009   |1990-01-22|1996-11-09|
|10012 |d005   |1992-12-18|9999-01-01|
|10013 |d003   |1985-10-20|9999-01-01|
|10014 |d005   |1993-12-29|9999-01-01|
|10015 |d008   |1992-09-19|1993-08-22|
|10016 |d007   |1998-02-11|9999-01-01|
|10017 |d001   |1993-08-03|9999-01-01|
|10018 |d004   |1992-07-29|9999-01-01|
|10018 |d005   |1987-04-03|1992-07-29|
+------+-------+----------+----------+
only showing top 20 rows



In [8]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.{row_number, rank, dense_rank, col}

// Operación de ventana: se ordenada de forma descendente por fecha todos los registros por empleado
// Se aprovecha de exploar las distitntas operaciones de ventana
val windowSpec = Window.partitionBy("emp_no").orderBy(col("to_date").desc)
val deptEmpSortedWithRowNumberDF = deptEmpDF.withColumn("row_number", row_number().over(windowSpec))
                                            .withColumn("rank", rank().over(windowSpec))
                                            .withColumn("dense_rank", dense_rank().over(windowSpec))

deptEmpSortedWithRowNumberDF.show(false)

+------+-------+----------+----------+----------+----+----------+
|emp_no|dept_no|from_date |to_date   |row_number|rank|dense_rank|
+------+-------+----------+----------+----------+----+----------+
|10206 |d005   |1988-04-19|9999-01-01|1         |1   |1         |
|10362 |d003   |1990-11-02|1997-07-16|1         |1   |1         |
|10623 |d008   |1996-02-11|9999-01-01|1         |1   |1         |
|10623 |d005   |1992-01-15|1996-02-11|2         |2   |2         |
|10817 |d009   |2000-01-24|9999-01-01|1         |1   |1         |
|10817 |d007   |1990-12-26|2000-01-24|2         |2   |2         |
|11033 |d005   |1991-03-14|9999-01-01|1         |1   |1         |
|11141 |d005   |1993-02-12|9999-01-01|1         |1   |1         |
|11317 |d005   |1998-07-04|9999-01-01|1         |1   |1         |
|11317 |d008   |1995-09-08|1998-07-04|2         |2   |2         |
|11458 |d008   |1985-10-06|9999-01-01|1         |1   |1         |
|11748 |d005   |1996-07-24|9999-01-01|1         |1   |1         |
|11858 |d0

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.{row_number, rank, dense_rank, col}
windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7a7da919
deptEmpSortedWithRowNumberDF: org.apache.spark.sql.DataFrame = [emp_no: int, dept_no: string ... 5 more fields]


In [9]:
// Nos quedamos con el registro mas reciente por empleado
// Aprovecha de eliminar las columnas adicionales
val lastDeptEmpSortedDF = deptEmpSortedWithRowNumberDF.where($"row_number" === 1)
                                                      .drop("row_number")
                                                      .drop("rank")
                                                      .drop("dense_rank")
lastDeptEmpSortedDF.show(false)

+------+-------+----------+----------+
|emp_no|dept_no|from_date |to_date   |
+------+-------+----------+----------+
|10206 |d005   |1988-04-19|9999-01-01|
|10362 |d003   |1990-11-02|1997-07-16|
|10623 |d008   |1996-02-11|9999-01-01|
|10817 |d009   |2000-01-24|9999-01-01|
|11033 |d005   |1991-03-14|9999-01-01|
|11141 |d005   |1993-02-12|9999-01-01|
|11317 |d005   |1998-07-04|9999-01-01|
|11458 |d008   |1985-10-06|9999-01-01|
|11748 |d005   |1996-07-24|9999-01-01|
|11858 |d002   |1996-08-15|9999-01-01|
|12027 |d008   |1992-01-05|9999-01-01|
|12046 |d001   |1996-03-12|1998-09-26|
|12799 |d003   |1993-12-25|9999-01-01|
|12940 |d005   |1987-12-12|9999-01-01|
|13285 |d005   |1997-06-25|2000-11-02|
|13289 |d001   |1987-09-24|9999-01-01|
|13623 |d006   |1995-10-29|9999-01-01|
|13832 |d004   |1999-01-19|1999-05-05|
|13840 |d005   |1998-12-12|1999-12-30|
|14450 |d005   |1985-04-26|1996-12-30|
+------+-------+----------+----------+
only showing top 20 rows



lastDeptEmpSortedDF: org.apache.spark.sql.DataFrame = [emp_no: int, dept_no: string ... 2 more fields]


In [10]:
// Utilizamos la misma operación de ventana para quedarnos con el salario más reciente de cada empelado
val lastSalariesDF = salariesDF.withColumn("row_number", row_number().over(windowSpec))
                              .where($"row_number" === 1)
                              .drop("row_number")

lastSalariesDF.show(false)

+------+------+----------+----------+
|emp_no|salary|from_date |to_date   |
+------+------+----------+----------+
|10206 |71052 |2002-04-16|9999-01-01|
|10362 |54987 |1996-10-31|1997-07-16|
|10623 |86399 |2002-01-12|9999-01-01|
|10817 |78202 |2001-12-23|9999-01-01|
|11033 |75271 |2002-03-11|9999-01-01|
|11141 |57809 |2002-02-10|9999-01-01|
|11317 |55544 |2001-09-06|9999-01-01|
|11458 |82731 |2001-10-02|9999-01-01|
|11748 |70230 |2002-07-23|9999-01-01|
|11858 |47998 |2001-08-14|9999-01-01|
|12027 |71281 |2001-11-19|9999-01-01|
|12046 |76043 |1998-01-12|1998-09-26|
|12799 |59831 |2001-12-23|9999-01-01|
|12940 |85425 |2001-12-08|9999-01-01|
|13285 |48494 |2000-06-24|2000-11-02|
|13289 |85488 |2001-09-20|9999-01-01|
|13623 |51145 |2001-10-27|9999-01-01|
|13832 |77530 |1998-10-05|1999-05-05|
|13840 |41453 |1999-12-12|1999-12-30|
|14450 |75419 |1996-04-23|1996-12-30|
+------+------+----------+----------+
only showing top 20 rows



lastSalariesDF: org.apache.spark.sql.DataFrame = [emp_no: int, salary: int ... 2 more fields]


In [11]:
// Hacemos join entre employeesDF y lastSalariesDF. De está forma enriquecemos el DF de empleados añadiendole a c/u su salario más reciente
// Esta forma de hacer el join se indica la expresión o condición de emparejamiento entre ambos DF
// EL hacerlo de esta forma hace que aparezcan todas las columnas de cada DF sin importar que se llamen igual y sean las utilizadas en la expresión del join
// Cuando se omite el tipo de join se asume que es un inner join
val employeesWithSalaryTempDF = employeesDF.join(lastSalariesDF, employeesDF("emp_no") === lastSalariesDF("emp_no"))
employeesWithSalaryTempDF.show()

+------+----------+----------+---------+------+----------+------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|salary| from_date|   to_date|
+------+----------+----------+---------+------+----------+------+------+----------+----------+
| 10206|1960-09-19|  Alassane|  Iwayama|     F|1988-04-19| 10206| 71052|2002-04-16|9999-01-01|
| 10362|1963-09-16|   Shalesh|  dAstous|     M|1988-08-24| 10362| 54987|1996-10-31|1997-07-16|
| 10623|1953-07-11|Aleksander|   Danlos|     F|1987-03-07| 10623| 86399|2002-01-12|9999-01-01|
| 10817|1958-10-02|       Uri|  Rullman|     F|1990-12-26| 10817| 78202|2001-12-23|9999-01-01|
| 11033|1957-03-01|   Shushma|     Bahk|     F|1990-10-02| 11033| 75271|2002-03-11|9999-01-01|
| 11141|1957-08-20|   Vasiliy|Kermarrec|     F|1989-12-28| 11141| 57809|2002-02-10|9999-01-01|
| 11317|1954-07-24|  Shigeaki| Hagimont|     F|1989-12-21| 11317| 55544|2001-09-06|9999-01-01|
| 11458|1958-08-09|     Stevo|Chenoweth|     F|198

employeesWithSalaryTempDF: org.apache.spark.sql.DataFrame = [emp_no: int, birth_date: date ... 8 more fields]


In [13]:
// Para evitar errores y duplicidad de datos nos quedamos con algunas columnas y escojemos una sola de cnúmero de empleado
// Por ello es necesario indicar a que DF pertenece la columna emp_no para evitar resolviendo el conflicto
val employeesWithSalaryDF = employeesWithSalaryTempDF.select(employeesDF("emp_no"), col("birth_date"), col("first_name"), col("last_name"), col("gender"), col("hire_date"), col("salary"))
employeesWithSalaryDF.show(false)

+------+----------+----------+---------+------+----------+------+
|emp_no|birth_date|first_name|last_name|gender|hire_date |salary|
+------+----------+----------+---------+------+----------+------+
|10206 |1960-09-19|Alassane  |Iwayama  |F     |1988-04-19|71052 |
|10362 |1963-09-16|Shalesh   |dAstous  |M     |1988-08-24|54987 |
|10623 |1953-07-11|Aleksander|Danlos   |F     |1987-03-07|86399 |
|10817 |1958-10-02|Uri       |Rullman  |F     |1990-12-26|78202 |
|11033 |1957-03-01|Shushma   |Bahk     |F     |1990-10-02|75271 |
|11141 |1957-08-20|Vasiliy   |Kermarrec|F     |1989-12-28|57809 |
|11317 |1954-07-24|Shigeaki  |Hagimont |F     |1989-12-21|55544 |
|11458 |1958-08-09|Stevo     |Chenoweth|F     |1985-10-06|82731 |
|11748 |1953-03-07|Lihong    |Massonet |M     |1992-12-20|70230 |
|11858 |1962-11-21|Slavian   |Baik     |M     |1988-11-12|47998 |
|12027 |1962-07-31|Zhanqiu   |Vuskovic |M     |1985-11-23|71281 |
|12046 |1961-05-16|Xiong     |Ranum    |F     |1996-01-13|76043 |
|12799 |19

employeesWithSalaryDF: org.apache.spark.sql.DataFrame = [emp_no: int, birth_date: date ... 5 more fields]


In [14]:
// Otra forma de llevar a cabo un Join para enriquecer el DF con el departamente del empleado 
// En este caso se indican el conj de columnas utilizadas como expresión del Join
// De esta forma se evita duplicidad de columnas especialmente en aquellas que forman parte del Join, por ejemplo emp_no
// Una forma de expresar el tipo de Join es mediante un String por ejemplo "inner"
val lastDeptWithNameEmpDF = departmentsDF.join(lastDeptEmpSortedDF, Seq("dept_no"), "inner")
            .select(departmentsDF.col("dept_no"), col("dept_name"), col("emp_no"), col("from_date"), col("to_date"))
lastDeptWithNameEmpDF.show(false)

+-------+------------------+------+----------+----------+
|dept_no|dept_name         |emp_no|from_date |to_date   |
+-------+------------------+------+----------+----------+
|d005   |Development       |10206 |1988-04-19|9999-01-01|
|d003   |Human Resources   |10362 |1990-11-02|1997-07-16|
|d008   |Research          |10623 |1996-02-11|9999-01-01|
|d009   |Customer Service  |10817 |2000-01-24|9999-01-01|
|d005   |Development       |11033 |1991-03-14|9999-01-01|
|d005   |Development       |11141 |1993-02-12|9999-01-01|
|d005   |Development       |11317 |1998-07-04|9999-01-01|
|d008   |Research          |11458 |1985-10-06|9999-01-01|
|d005   |Development       |11748 |1996-07-24|9999-01-01|
|d002   |Finance           |11858 |1996-08-15|9999-01-01|
|d008   |Research          |12027 |1992-01-05|9999-01-01|
|d001   |Marketing         |12046 |1996-03-12|1998-09-26|
|d003   |Human Resources   |12799 |1993-12-25|9999-01-01|
|d005   |Development       |12940 |1987-12-12|9999-01-01|
|d005   |Devel

lastDeptWithNameEmpDF: org.apache.spark.sql.DataFrame = [dept_no: string, dept_name: string ... 3 more fields]


In [15]:
import org.apache.spark.sql.catalyst.plans._

// Otra forma de expresar el Join es mediante unas constantes 
// de esta forma se puede evitar errores en tiempo de ejecución al indicar mal un tipo de Join
val finalEmployeesEnrichedDF = employeesWithSalaryDF.join(lastDeptWithNameEmpDF, Seq("emp_no"), Inner.sql)
finalEmployeesEnrichedDF.show(false)

+------+----------+----------+---------+------+----------+------+-------+------------------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender|hire_date |salary|dept_no|dept_name         |from_date |to_date   |
+------+----------+----------+---------+------+----------+------+-------+------------------+----------+----------+
|10206 |1960-09-19|Alassane  |Iwayama  |F     |1988-04-19|71052 |d005   |Development       |1988-04-19|9999-01-01|
|10362 |1963-09-16|Shalesh   |dAstous  |M     |1988-08-24|54987 |d003   |Human Resources   |1990-11-02|1997-07-16|
|10623 |1953-07-11|Aleksander|Danlos   |F     |1987-03-07|86399 |d008   |Research          |1996-02-11|9999-01-01|
|10817 |1958-10-02|Uri       |Rullman  |F     |1990-12-26|78202 |d009   |Customer Service  |2000-01-24|9999-01-01|
|11033 |1957-03-01|Shushma   |Bahk     |F     |1990-10-02|75271 |d005   |Development       |1991-03-14|9999-01-01|
|11141 |1957-08-20|Vasiliy   |Kermarrec|F     |1989-12-28|57809 |d005   |Develop

import org.apache.spark.sql.catalyst.plans._
finalEmployeesEnrichedDF: org.apache.spark.sql.DataFrame = [emp_no: int, birth_date: date ... 9 more fields]


In [16]:
println(employeesDF.count)
println(finalEmployeesEnrichedDF.count)

300024
300024
